In [3]:
import os
import pandas as pd
import json
import traceback

In [4]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
key=os.getenv('api_key')

In [10]:
print(key)

None
